# MobileNetV3

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
model = models.mobilenet_v3_small(pretrained=False)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
num_params = sum(p.numel() for p in model.parameters())
num_params

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


1528106

# RESNET

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
num_params = sum(p.numel() for p in model.parameters())
num_params

11181642

# ALEX NET

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
model = models.alexnet(pretrained=False)
model.features[0] = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
num_params = sum(p.numel() for p in model.parameters())
num_params

57023306

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CIFAR10ModifiedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10ModifiedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.3)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, padding=2)  # 5x5 kernel
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout(0.3)

        self.conv5 = nn.Conv2d(64, 128, kernel_size=5, padding=2)  # 5x5 kernel
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout(0.3)

        self.fc1 = nn.Linear(128 * 4 * 4, 1024)
        self.bn_fc1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn_fc2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, num_classes)

        self.residual = nn.Conv2d(32, 128, kernel_size=1, stride=4, padding=0)

    def mish(self, x):
        return x * torch.tanh(F.softplus(x))

    def forward(self, x):
        x = self.mish(self.bn1(self.conv1(x)))
        x = self.mish(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)

        identity = self.residual(x)

        x = self.mish(self.bn3(self.conv3(x)))
        x = self.mish(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)

        x = self.mish(self.bn5(self.conv5(x)))
        x = self.mish(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)

        x += identity

        x = x.view(-1, 128 * 4 * 4)

        x = self.mish(self.bn_fc1(self.fc1(x)))
        x = self.mish(self.bn_fc2(self.fc2(x)))
        x = self.fc3(x)

        return x

import torch
import torch.nn as nn
from torchvision import models

model = CIFAR10ModifiedCNN(num_classes=10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
num_params = sum(p.numel() for p in model.parameters())

num_params

3087146